In [1]:
import pandas as pd
import numpy as np
import random

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

import rdkit
from rdkit.Chem import AllChem, rdFingerprintGenerator
from rdkit import Chem, DataStructs

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

import matplotlib.pyplot as plt

import catboost as cat

c:\Users\BSH\.virtualenvs\신약개발-mImSxWNo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
RANDOM_SEED = 777

In [3]:
def feature_engineer():
  train_df = pd.read_csv("../data/train.csv")
  test_df = pd.read_csv("../data/test.csv")
  train_df = train_df.dropna()
  test_df["AlogP"] = np.where(pd.isna(test_df["AlogP"]), test_df["LogD"], test_df["AlogP"])
  
  train_df["mol"] = train_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  test_df["mol"] = test_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  
  train_df.drop(columns=["id", "SMILES"], inplace=True)
  test_df.drop(columns=["id", "SMILES"], inplace=True)
  
  fmgen = rdFingerprintGenerator.GetMorganGenerator()
  train_fps = train_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  train_fps = np.stack(train_fps)
  test_fps = test_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  test_fps = np.stack(test_fps)
  
  origin_train_features = train_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea"]].values
  origin_test_features = test_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea"]].values
  
  train_features = np.append(origin_train_features, train_fps, axis=1)
  test_features = np.append(origin_test_features, test_fps, axis=1)
  target = train_df["MLM"].values
  
  return train_features, target, test_features
  

In [4]:
train_x, train_y, test_x = feature_engineer()

In [5]:
def create_model(param):
  return cat.CatBoostRegressor(
    random_state=RANDOM_SEED,
    verbose=False,
    **param
  )

In [6]:
def train_model(param, X, y, X_test, trial=None):
    skf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

    val_scores = []
    y_tests = []
    models = []

    for idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):

        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]

        model = create_model(param)
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=50, verbose=500)

        y_hat_val = model.predict(X_val)
        score = mean_squared_error(y_val, y_hat_val, squared=True)
        val_scores.append(score)
        print(f'Fold: {idx+1}/5 score = {score:.5f}')

        y_tests.append(model.predict(X_test))
        models.append(model)

        if trial:
            trial.report(score, idx)

            if trial.should_prune():
                raise optuna.TrialPruned()

    return val_scores, y_tests, models

In [7]:
def objective_cat(trial):
    param = {
        "iterations": trial.suggest_int("iterations", 2000, 20000),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
    }

    val_scores, y_tests, models = train_model(param, train_x, train_y, test_x, trial)

    return sum(val_scores) / len(val_scores)

In [8]:
study = optuna.create_study(
    sampler=TPESampler(seed=RANDOM_SEED),
    direction='minimize',
    study_name='cat_tuning',
    pruner=HyperbandPruner(
        min_resource=1, max_resource=8, reduction_factor=3
    ),
)

study.optimize(objective_cat, n_trials=70)

[I 2023-09-06 19:11:19,134] A new study created in memory with name: cat_tuning


0:	learn: 35.2077973	test: 35.2077973	test1: 35.7341293	best: 35.7341293 (0)	total: 136ms	remaining: 10m 47s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.85583915
bestIteration = 212

Shrink model to first 213 iterations.
Fold: 1/5 score = 1079.50617
0:	learn: 35.4333864	test: 35.4333864	test1: 34.9403593	best: 34.9403593 (0)	total: 1.86ms	remaining: 8.85s
500:	learn: 29.7004173	test: 29.7004173	test1: 30.6308328	best: 30.6095808 (486)	total: 675ms	remaining: 5.73s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.53938104
bestIteration = 586

Shrink model to first 587 iterations.
Fold: 2/5 score = 932.65380
0:	learn: 35.4350180	test: 35.4350180	test1: 34.9615243	best: 34.9615243 (0)	total: 1.4ms	remaining: 6.65s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.93455141
bestIteration = 303

Shrink model to first 304 iterations.
Fold: 3/5 score = 956.94647
0:	learn: 35.2772775	test: 35.2772775	test1: 35.5533315	best: 35.5533

[I 2023-09-06 19:11:37,049] Trial 0 finished with value: 992.9128991512587 and parameters: {'iterations': 4748, 'learning_rate': 0.09140462630254884, 'depth': 1, 'subsample': 0.5138743074480719, 'min_data_in_leaf': 86, 'colsample_bylevel': 0.9306471953623999}. Best is trial 0 with value: 992.9128991512587.


0:	learn: 34.3668764	test: 34.3668764	test1: 35.0684135	best: 35.0684135 (0)	total: 3.06ms	remaining: 46.2s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.358546
bestIteration = 125

Shrink model to first 126 iterations.
Fold: 1/5 score = 1047.07550
0:	learn: 34.9195744	test: 34.9195744	test1: 34.3467969	best: 34.3467969 (0)	total: 3.13ms	remaining: 47.3s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.19347206
bestIteration = 221

Shrink model to first 222 iterations.
Fold: 2/5 score = 911.64576
0:	learn: 35.4321479	test: 35.4321479	test1: 34.9946349	best: 34.9946349 (0)	total: 3.12ms	remaining: 47.1s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.67625478
bestIteration = 133

Shrink model to first 134 iterations.
Fold: 3/5 score = 941.03261
0:	learn: 35.3953956	test: 35.3953956	test1: 35.7988009	best: 35.7988009 (0)	total: 3.13ms	remaining: 47.2s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.29152

[I 2023-09-06 19:11:54,976] Trial 1 finished with value: 967.2132460117103 and parameters: {'iterations': 15086, 'learning_rate': 0.2307803703542728, 'depth': 3, 'subsample': 0.6796263628638904, 'min_data_in_leaf': 18, 'colsample_bylevel': 0.1257015913309088}. Best is trial 1 with value: 967.2132460117103.


...

Fold: 2/5 score = 918.00799
0:	learn: 35.1493599	test: 35.1493599	test1: 35.6610478	best: 35.6610478 (0)	total: 19.7ms	remaining: 5m
Stopped by overfitting detector  (50 iterations wait)

bestTest = 32.24181088
bestIteration = 270

Shrink model to first 271 iterations.
Fold: 1/5 score = 1039.53437
0:	learn: 35.3549116	test: 35.3549116	test1: 34.8746589	best: 34.8746589 (0)	total: 19.3ms	remaining: 4m 54s
500:	learn: 14.5031516	test: 14.5031516	test1: 30.0164920	best: 29.9643727 (450)	total: 11.3s	remaining: 5m 32s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 29.96437273
bestIteration = 450

Shrink model to first 451 iterations.
Fold: 2/5 score = 897.86364
0:	learn: 35.3980831	test: 35.3980831	test1: 34.9675202	best: 34.9675202 (0)	total: 19.8ms	remaining: 5m 1s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 29.9753839
bestIteration = 290

Shrink model to first 291 iterations.
Fold: 3/5 score = 898.52364
0:	learn: 35.1185968	test: 35.1185968	test1

[I 2023-09-06 19:50:30,622] Trial 68 finished with value: 942.3660022774051 and parameters: {'iterations': 15245, 'learning_rate': 0.08518476546941436, 'depth': 8, 'subsample': 0.4770926242431474, 'min_data_in_leaf': 64, 'colsample_bylevel': 0.7319432084883337}. Best is trial 49 with value: 937.657411773767.


Fold: 5/5 score = 924.34784
0:	learn: 35.1141232	test: 35.1141232	test1: 35.6403021	best: 35.6403021 (0)	total: 19ms	remaining: 4m 56s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.91951892
bestIteration = 176

Shrink model to first 177 iterations.
Fold: 1/5 score = 1018.85569
0:	learn: 35.2307510	test: 35.2307510	test1: 34.8172953	best: 34.8172953 (0)	total: 19.4ms	remaining: 5m 2s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.1492343
bestIteration = 197

Shrink model to first 198 iterations.
Fold: 2/5 score = 908.97633
0:	learn: 35.3797455	test: 35.3797455	test1: 34.8535550	best: 34.8535550 (0)	total: 19.7ms	remaining: 5m 6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.16727263
bestIteration = 294

Shrink model to first 295 iterations.
Fold: 3/5 score = 910.06434
0:	learn: 35.2209739	test: 35.2209739	test1: 35.5447040	best: 35.5447040 (0)	total: 19.5ms	remaining: 5m 3s
Stopped by overfitting detector  (50 iteration

[I 2023-09-06 19:51:20,186] Trial 69 finished with value: 942.2297543300307 and parameters: {'iterations': 15572, 'learning_rate': 0.0901969980457376, 'depth': 8, 'subsample': 0.48989875014533324, 'min_data_in_leaf': 65, 'colsample_bylevel': 0.8440749231913413}. Best is trial 49 with value: 937.657411773767.


Fold: 5/5 score = 920.46742


In [9]:
trial = study.best_trial

In [10]:
print(trial.params)

{'iterations': 9754, 'learning_rate': 0.11952641635441928, 'depth': 10, 'subsample': 0.31744490157253696, 'min_data_in_leaf': 51, 'colsample_bylevel': 0.5742054173224616}


In [11]:
best_param = {
  'iterations': 9754,
  'learning_rate': 0.11952641635441928,
  'depth': 10,
  'subsample': 0.31744490157253696,
  'min_data_in_leaf': 51,
  'colsample_bylevel': 0.5742054173224616
}
val_scores, y_tests, models = train_model(best_param, train_x, train_y, test_x)

0:	learn: 34.8374969	test: 34.8374969	test1: 35.5981129	best: 35.5981129 (0)	total: 66.1ms	remaining: 10m 44s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 31.96114988
bestIteration = 316

Shrink model to first 317 iterations.
Fold: 1/5 score = 1021.51511
0:	learn: 35.0988496	test: 35.0988496	test1: 34.7864720	best: 34.7864720 (0)	total: 64.5ms	remaining: 10m 28s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 29.8354013
bestIteration = 180

Shrink model to first 181 iterations.
Fold: 2/5 score = 890.15117
0:	learn: 35.0710486	test: 35.0710486	test1: 34.5890243	best: 34.5890243 (0)	total: 72.8ms	remaining: 11m 49s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 30.25840019
bestIteration = 94

Shrink model to first 95 iterations.
Fold: 3/5 score = 915.57078
0:	learn: 34.9207218	test: 34.9207218	test1: 35.3800721	best: 35.3800721 (0)	total: 65.5ms	remaining: 10m 38s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 3

In [12]:
for idx, model in enumerate(models):
  model.save_model(f"model/catboost_MLM_fold_{idx}.model")

In [13]:
submit = pd.read_csv("./catboost_optuna.csv")
submit["MLM"] = np.mean(y_tests, axis=0)
submit.to_csv("catboost_optuna.csv", index=False)